# Capstone Project - Night Markets of Darwin region in Australia 

## Introduction: Night Market Development

Darwin is the capital city of the Northern Territory, Australia. **Greater Darwin region** are with a 2019 estimated **population of 147,255** it has more residents than the rest of the sparsely populated Northern Territory. It is the smallest, wettest, and most northerly of the Australian capital cities and serves as the Top End's regional centre.
Greater Darwin includeing capital Darwin city region and Palmerston city mainly which **Darwin** is with a population of **82,886** and **Palmerston** is **38,270.**

This smallest city is the only city with night markets in Australia because it inspired by South Asia, also the location that Darwin as Asia's gateway to Australia. According to the tourism summary, there were **4,473,806 visitor nights** in Greater Darwin region in 2019 last year that is a tremendous number for a small region. 

**In this project, we are going to find out that do we need more night market? If so, where will it be? or the market is full now?**




### References
Wiki https://en.wikipedia.org/wiki/Darwin,_Northern_Territory

NT populatin https://nteconomy.nt.gov.au/population

Tourism visitor summary https://economy.id.com.au/darwin/tourism-visitor-summary?BMID=20

City of Palmerston LGA https://profile.id.com.au/rda-northern-territory/about?WebID=200


## Data Section

### Import libraries

In [1]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import pandas as pd 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from pandas.io.json import json_normalize  

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Access Foursquare

In [5]:
CLIENT_ID = 'ZRMZZGCXZLQBD0HE52LWZQ1JKLTZFTQF545O0SYT213KFBOK' # your Foursquare ID
CLIENT_SECRET = 'TULJH33RBUTZ3E4NV4LVRHC4O4JE425WUIQP2FVG41N4P04P' # your Foursquare Secret
ACCESS_TOKEN = 'RTHUEBRBCVGK4EZDSF051F4MTWRUMWUCAVZIAW0YORDHQLMP' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

### Get the markets data

In [6]:
address = 'Darwin city, Australia'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-12.460834 130.844442


In [7]:
search_query = 'market'
radius = 40000
print(search_query + ' .... OK!')

market .... OK!


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ZRMZZGCXZLQBD0HE52LWZQ1JKLTZFTQF545O0SYT213KFBOK&client_secret=TULJH33RBUTZ3E4NV4LVRHC4O4JE425WUIQP2FVG41N4P04P&ll=-12.460834,130.844442&oauth_token=RTHUEBRBCVGK4EZDSF051F4MTWRUMWUCAVZIAW0YORDHQLMP&v=20180604&query=market&radius=40000&limit=30'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fe1d3f11539512b520960f4'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4e5afd181f6e804280daa7a1',
    'name': 'Darwin Fish Market',
    'location': {'lat': -12.456999,
     'lng': 130.846864,
     'labeledLatLngs': [{'label': 'display',
       'lat': -12.456999,
       'lng': 130.846864}],
     'distance': 501,
     'cc': 'AU',
     'state': 'Northern Territory',
     'country': 'Australia',
     'formattedAddress': ['Darwin City NT']},
    'categories': [{'id': '4d4b7105d754a06378d81259',
      'name': 'Shop & Service',
      'pluralName': 'Shops & Services',
      'shortName': 'Shops',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1608635377',
    'hasPerk': False},
   {'id': '55419014498e544ab99b7b25',
    'name': 'Mindil Beach Sunset Market Association',
    'location': {'

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,state,country,formattedAddress,address,postalCode,city,id
0,Darwin Fish Market,Shop & Service,-12.456999,130.846864,"[{'label': 'display', 'lat': -12.456999, 'lng'...",501,AU,Northern Territory,Australia,[Darwin City NT],NaN,NaN,NaN,4e5afd181f6e804280daa7a1
1,Mindil Beach Sunset Market Association,Non-Profit,-12.454504,130.836639,"[{'label': 'display', 'lat': -12.454504, 'lng'...",1102,AU,NaN,Australia,NaN,NaN,NaN,NaN,55419014498e544ab99b7b25
2,Greek Market Festival,None,-12.454586,130.831403,"[{'label': 'display', 'lat': -12.4545861, 'lng...",1578,AU,NaN,Australia,NaN,NaN,NaN,NaN,593d31c98c812a1b83d48c42
3,Mindil Beach Markets,Farmers Market,-12.444942,130.831934,"[{'label': 'display', 'lat': -12.4449422632791...",2231,AU,NT,Australia,"[Maria Liveris Dr, Darwin NT 0820]",Maria Liveris Dr,0820,Darwin,4befb148f831c9280d3d01f2
4,Nightcliff Market,Market,-12.387317,130.850835,"[{'label': 'display', 'lat': -12.38731742, 'ln...",8213,AU,NT,Australia,"[Pavonia Place, Nightcliff NT 0810]",Pavonia Place,0810,Nightcliff,51943b1d498ea6c38d4da6d6
5,Fresh Produce Market,Gourmet Shop,-12.398511,130.855081,"[{'label': 'display', 'lat': -12.398511, 'lng'...",7033,AU,Northern Territory,Australia,"[50 De Latour St., Coconut Grove NT 0810]",50 De Latour St.,0810,NaN,50b041fbe4b0e7b98ef89bbd
6,Parap Village Markets,Farmers Market,-12.431360,130.844183,"[{'label': 'display', 'lat': -12.4313601635266...",3281,AU,NT,Australia,"[Parap Rd, Darwin NT]",Parap Rd,NaN,Darwin,4be4daaa2457a593b6e6aa15
7,Malak Marketplace,None,-12.396980,130.903610,"[{'label': 'display', 'lat': -12.39698, 'lng':...",9586,AU,NaN,Australia,NaN,NaN,NaN,NaN,59045bdca4b51b124bfd0ba8
8,Palmerston Rural Market,None,-12.479895,130.984616,"[{'label': 'display', 'lat': -12.4798953, 'lng...",15382,AU,NaN,Australia,NaN,NaN,NaN,NaN,59b286a76eda027f5426724c
9,Palmerston Friday Market,None,-12.480255,130.985140,"[{'label': 'display', 'lat': -12.4802547, 'lng...",15444,AU,NaN,Australia,NaN,NaN,NaN,NaN,53884dce498e8da3bcc31560


### Keep only Market, remove supermarket stores

In [17]:
market_df = dataframe_filtered[dataframe_filtered['categories'].str.contains("Market", na=False)]
market_df

,name,categories,lat,lng,labeledLatLngs,distance,cc,state,country,formattedAddress,address,postalCode,city,id
3,Mindil Beach Markets,Farmers Market,-12.444942,130.831934,"[{'label': 'display', 'lat': -12.4449422632791...",2231,AU,NT,Australia,"[Maria Liveris Dr, Darwin NT 0820]",Maria Liveris Dr,0820,Darwin,4befb148f831c9280d3d01f2
4,Nightcliff Market,Market,-12.387317,130.850835,"[{'label': 'display', 'lat': -12.38731742, 'ln...",8213,AU,NT,Australia,"[Pavonia Place, Nightcliff NT 0810]",Pavonia Place,0810,Nightcliff,51943b1d498ea6c38d4da6d6
6,Parap Village Markets,Farmers Market,-12.431360,130.844183,"[{'label': 'display', 'lat': -12.4313601635266...",3281,AU,NT,Australia,"[Parap Rd, Darwin NT]",Parap Rd,NaN,Darwin,4be4daaa2457a593b6e6aa15
11,Rapid Creek Markets,Farmers Market,-12.387076,130.862688,"[{'label': 'display', 'lat': -12.3870759923626...",8446,AU,NT,Australia,"[48 Trower Rd, Milner NT]",48 Trower Rd,NaN,Milner,4c9e92a1542b224b9ca0f59f
12,Palmerston Markets,Flea Market,-12.479847,130.984500,"[{'label': 'display', 'lat': -12.4798465, 'lng...",15369,AU,NT,Australia,"[6 The Boulevard, Palmerston City NT 0830]",6 The Boulevard,0830,Palmerston City,4c65027ddddfa593de1493ff
13,Berry Springs Markets,Flea Market,-12.710838,130.999476,"[{'label': 'display', 'lat': -12.7108381224039...",32530,AU,NaN,Australia,NaN,NaN,NaN,NaN,4f6e8ed7e4b0ac4cdb62cc4e


### Shape

In [18]:
market_df.shape

(6, 14)

#### We have six nightmarket. Let's look where are they.

### Map

In [36]:
latitude = -12.479847 
longitude = 130.984500

# Create a map
market_map = folium.Map(location=[latitude, longitude], zoom_start=9)


markets = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(market_df.lat, market_df.lng):
    markets.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=True,
            fill_color='yellow',
            fill_opacity=0.6
        )
    )

# add incidents to map
market_map.add_child(markets)

#### Two night markets and two morning markets in Darwin
#### One night market in Palmerston
#### One morning market in Berry Springs. 

###  There are only 818 people living in Berry Springs region,  we can image they are happy with a market. And morning markets are common markets in many countries, so we only look at the night markets.

### Darwin vs Palmerston

In [42]:
data = [["Darwin","Mindil, Nightcliff", 2, 82886],["Palmerston","Palmerston", 1, 38270]]
market2_df = pd.DataFrame(data, columns=["Region", "Market Name", "Market Number", "Population"])

market2_df


,Region,Market Name,Market Number,Population
0,Darwin,"Mindil, Nightcliff",2,82886
1,Palmerston,Palmerston,1,38270


In [43]:
market2_df.insert(4,"Density of Population per Market", market2_df['Population'] / market2_df['Market Number'], True)
market2_df

,Region,Market Name,Market Number,Population,Density of Population per Market
0,Darwin,"Mindil, Nightcliff",2,82886,41443.0
1,Palmerston,Palmerston,1,38270,38270.0


### Darwin and Palmerston are very small cities, hot and monson in the wet season, easily over 40 degrees outside and raining frequently, so unfortunate, most night markets are close during the wet season. Even we knew there are a huge amount of visitors, but looks likely **the market is full now**.

### This report with a **deeper analysis potential**, including how many people usually go to the night market? how's the average income related to the probability that people would go to the night market? Or visitors' background? For now, I don't have enough dataset to analysis further. Hope it's enough for the assignment. **Thank you for your reading.**
